In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install Unidecode

     |████████████████████████████████| 245kB 3.2MB/s 


In [ ]:
# Imports

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np
import pandas as pd

import random
import re
import math
import time

import spacy
import torchtext
from torchtext.legacy.data import Field,BucketIterator,TabularDataset
from sklearn.model_selection import train_test_split



In [ ]:
torch.backends.cudnn.deterministic = True

# Data Preprocessing

In [ ]:
# replacing contraction words

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",'i\'m':'i am', "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [ ]:
# dealing with spl characters
punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';',  '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√','∞','θ','÷','α','•','à','−','β','∅','³','π','‘','₹','´','°','™','√²','—–' ]

In [ ]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [ ]:
missplet_dict = {'didnt':'did not',
 "i'm": 'i am',
 'doesnt' : 'does not',
 'isnt' : 'is not',
 'upvote' : 'up vote',
 'wasnt' : 'was not',
 'remindme': 'remind me',
 'lt3' : 'love',
 'upvotes' : 'up votes',
 'shouldnt' : 'should not',
 'hasnt' : 'has not',
 'downvoted' : 'down voted',
 'howd' : ' how do you do',
 'upvoted' : 'up voted',
 'rcasualconversation' : 'casual conversation',
 'cakeday' : 'cake day',
 'downvote' : 'down vote',
 'whatre' : 'what are',
 'contenderwhat' : 'contender what',
 'downvotes' : 'down votes',
 'heshe' : 'he she',
 'tldr' : 'too long did not read',
 'ftfy' : 'Fixed That For You',
 'wbu' : 'what about you',
 'thatd' : 'that would',
 'welp' : 'well',
 'ikr' : 'i know right',
 'realise' : 'realize',
 'thatll' : 'that will',
 'upvoting' : 'up voting',
 'whatd' : ' what did',
 'grey': 'gray',
 'yesno' : 'yes no',
 'flavour' : 'flavor',
 'tooslowly' : 'too slowly',
 'realised' : 'realized',
 'learnt' : 'learn',
 'answers—comment' :  'answers comment',
 'gorillaz' : 'gorilla',
 '“fuck' : 'fuck',
 'accomplishedbeen' : 'accomplished been',
 'whyd' : 'why did',
 'edm' : 'electronic dance music',
 'linkin' : 'linking',
 'hbu' : 'how about you',
 'mustve' : 'must have',
 'whered' : 'where did',
 'continuein' : 'continuing',
 'welldo' : 'well done',
 'downvoter' : 'down voter',
 'friendo' : 'friend',
 '“fixed' : 'fixed',
 'yknow' : 'you know',
 'programme' : 'program',
 'heya' : 'hi',
 'neighbours' : 'neighbor',
 'hmu' : 'hit me up',
 'twinsies' : 'twins',
 '“what' : 'what',
 'inb4' : 'in before',
 'halflife' : 'half life',
 'heyy' : 'hey',
 'summarise' : 'summarize',
 'lpt' : 'life pro tip',
 'gofundme' : 'go find me',
 '5yearold' : '5 year old',
 'noyou' : 'no you',
 'yesyou' : 'yes you',
 'life".' : 'life',
 'yess' : 'yes',
 'greys' : 'gray',
 'daynight' : 'day night',
 'yessss' : 'yes',
 'trickortreating' : 'trick or treat',
 'ninenine' : 'nine nine',
 'dnd' : 'do not disturb',
 'howre' : 'how are',
 'highfive' : 'high five',
 'favour' : 'favor',
 'gtfo' : 'Get the fuck out',
 'necessarythey' : 'necessary they',
 'yessssss' : 'yes',
 'jaja' : 'haha',
 'stopdinner' : 'stop dinner',
 'wristbander' :  'wrist bander',
 'tf2' : 'team fortress',
 'ptsd' : 'post traumatic stress disorder',
 'whataburger' : 'what a burger',
 'youi' : 'you i',
 'nosleep' : 'no sleep',
 'warframe' : 'war frame',
 'what?*' : 'what',
 'yoooo' : 'yes',
 'lul' : 'love you lots',
 'pmed' : 'private messaged',
 ':&gt' : 'get through',
 'pmd' : 'private messaged',
 'ahahah' : 'haha',
 'rsuddenlygay' : 'suddenly gay',
 'rsuicidebywords' : 'suicide by words',
 'rmadlads' :'mad lads',
 '“oh' : 'oh',
 'i‘m' : 'i am',
 'raww' : 'raw',
 'clickbait' : 'click bait',
 'rshowerthoughts' : 'shower thoughts',
 'ymmv' : 'your mileage may vary',
 'imnsho' : 'in my not so humble opinion',
 'shittttt' : 'shit',
 'holllyyyyyyy' : 'holy',
 'foxygen' : 'oxygen',
 'rteenagers' : 'teenagers',
 'gotchu' : 'got you',
 'boredwho' : 'bored who',
 'gt;: ' : 'get through',
 'coldplay' : 'cold play',
 'crosspost' : 'cross post',
 ' #tellyourstoryin5words' : 'tell your story in 5 words',
 'isare' : 'is are',
 'sfw' : 'safe for work',
 'downvoting' :  'down voting',
 'rpointlessstories' : 'pointless stories',
 'now?!' : 'now',
 'apologise' : 'apologize',
 '“they' : 'they',
 'enough’.' : 'enough',
 'goodcool' : 'good cool',
 'gtgt': 'got to go to',
 '‘not' : 'not',
 'upvoter' : 'up voter',
 'breathtakingyour' :  'breath taking our',
 'runexpectedoffice' : 'unexpected office',
 'leavingbi' : 'leaving',
 'rfoundthemobileuser' : 'found the mobile user',
 'thanksif' : 'thanks if',
 'rwholesome' : 'wholesome',
 'lol' : 'laugh out loud',
 'rlounge' : 'lounge',
 'soundcloud' : 'sound cloud',
 'leppard' : 'leopard',
 'rtotallynotrobots' : 'totally not robots',
 'cavetown' : 'cave town',
 'blinddisabled' : 'blind disabled',
 'introverting' : 'introvert',
 'lmaoooo' : 'laughing my ass off',
 'yh' : 'yes',
 'sucks!"america' : 'sucks america',
 'fawlty' : 'faulty',
 'runpopularopinion' : 'run popular opinion',
 'yesssss' : 'yes',
 'crossdress' : 'cross dress',
 'adminsemployees' : 'admins employees',
 'smokedis' : 'smoke',
 'spelt' : 'spell',
 '“far' : 'far',
 'dayweekmonth' : 'day week month',
 'thicc' : 'thick',
 'yaaay' : 'yes',
 'suckswhats' : 'sucks what',
 'durnk' : 'drunk',
 'butterfinger' : 'butter finger',
 'tekken' : 'taken',
 'yayyyy' : 'yes',
 'recall': 'recall',
 'lotr' : 'lord of the rings',
 'mhm' : 'yes',
 'travelled' : 'travel',
 'lolyou' : 'laugh out loud you',
 'rlifeprotips' : 'life pro tips',
 'itwhat' : 'it what',
 'rwallstreetbets' : 'wall street bets',
 'sharptooth' : 'sharp tooth',
 'vsauce' : 'sauce',
 'yayyy' : 'yes',
 'casualconversation' : 'casual conversation',
 'world war 2' : 'world war 2',
 'hiiii' : 'hi',
 'no3' : 'no',
 'selfie' : 'selfie',
 'dreamworks' : 'dream works',
 'bf4' : 'before',
 'hej' : 'hi',
 'flavoured' : 'flavored',
 'daaamn' : 'damn',
 'son' : 'son',
 'you' : 'you',
 'gay".' : 'gay',
 'nword' : 'word',
 'rthedavincicode' : 'the vinci code',
 'brexit' : 'brexit',
 'midtwenties' : 'mid twenties',
 'bingewatch' : 'binge watch',
 'peel”.' : 'peel',
 'yourselfwell' : 'yourself well',
 'yppah' : 'yes',
 'rbisexual' : 'bisexual',
 'kindda' : 'kind of',
 'runderratedcomments' : 'underrated comments',
 'backmaybe' : 'back may be',
 'itbut' : 'it but',
 'suggestionsi' : 'suggestions',
 'motherfuckerglad' : 'mother fucker glad',
 'nvm' : 'never mind',
 'songshelp' : 'songs help',
 'positivityfun' : 'positivity fun',
 'indoorone' : 'indore one',
 'timehow' : 'time how',
 'lolbut' : 'laugh out loud but',
 'manwoman' : 'man woman',
 'feelingdoing' : 'feeling doing',
 'friendscould' : 'friends cloud',
 'rhappyrelationships' : 'happy relationships',
 'do. Lonely' : 'do lonely',
 'realisation' : 'realization',
 'woooow' : 'wow',
 'rcatsstandingup' : 'cats standing up',
 'goldenboye' : 'golden boy',
 'weeki' : 'week',
 'numbah' : 'number',
 'yeahh' : 'yes',
 'youim' : 'you i am',
 'toowhat' : 'too what',
 'deffinetly' : 'definitely',
 'game3' : 'game',
 'xpost' : 'post',
 'rchildfree' : 'child free',
 'music2' : 'music',
 'quotebook' : 'quote book',
 'ufuckswithducks' : 'fuck with duck',
 'lold' : 'old',
 'whaaaaat' : 'what',
 'flavours' : 'flavors',
 'rpics' : 'pictures',
 'piano2' : 'piano',
 'requirementsi' : 'requirements',
 'himher' : 'him her',
 'yeaaah' : 'yes',
 'rsuicidewatch' : 'suicide watch',
 'damnnnn' : 'damn',
 'longsword' : 'long word',
 'awfuli' : 'awefully',
 'humour' : 'humor',
 'fooddrink' : 'food drink',
 'hby' : 'how about you',
 'roomwhat' : 'room what',
 'selfies' : 'selfie',
 'feelsbadman' : 'feels bad man',
 'youuu' : 'you',
 'mustnt' : 'must not',
 'nederlands' : 'netherlands',
 'aaaaand' : 'and',
 'smoll' : 'small',
 'yus' : 'yes',
 'madlad' : 'mad lad',
 'rniceguys' : 'nice guys',
 'uyourearealcunt' : ' you are a real cunt',
 'rimsorryjon' : 'i am sorry jon',
 'yet."' : 'yet',
 'edit😬' : 'edit',
 'retreival' : 'retrieval',
 'haaaaate' : 'hate',
 'musicianband' : 'musician band',
 'wingies' : 'wings',
 'money4' : 'money',
 'work2' : 'work',
 'goodi' : 'good',
 'touchstarved' : 'touch starved',
 'grindr' : 'grinder',
 'rblunderyears' : 'blunder years',
 'rcursedcomments' : 'cursed comments',
 '‘em' : 'them',
 'rstaticsnake' : 'static snake',
 'marcy' : 'mercy',
 'mightve' : 'might have',
 'cuteits' : 'cute',
 'whatim' : 'what i am',
 'havingfun' : 'having fun',
 'exwifes' : 'ex wives',
 'noyoure' : 'no you are',
 'aaaaand' : 'and',
 'smoll' : 'small',
 'yus' : 'yes',
 "i'hv": "i have",
 "i'll": "i will",
 '☺️':'😊',
 'ampx###b': 'amp bitch',"i've": 'i have','rwholesomememes':'wholesome memes',
 'hello😀😊☺🙌👍':'hello 😀 😊','͜ʖ': '😊', '😂😂':'😂','😂😂😂':'😂','ayyyy':'yes','\U0001f970':'😊', '🅱️ailure' : 'big failure', "goin'": 'going','👉😎👉': '😎', "d'aww" : 'so cute','ulionghost': ' you lion ghost',"fuckin'": 'fucking', 'uiwinalot7': ' you i win a lot', 'goodra': 'good',
'pie5': 'being soft', '😀😊☺': '😀', 'uwaterguy##': ' you water guy', ' rseriousconversation': 'serious conversation','runexpectedfactorial':'unexpected factorial', "i'mma": 'i am',
'👏👏':'👏', "lol'd": "laughed out loud", 'ಠಠ': '😎', 'rtea': 'tea', '🤣🤣🤣': '🤣','👏👏👏':'👏',"you'": 'you', "d'aw": 'so cute', 'pmmeyogurtpics':'send me yogurt pictures',
"sirma'am": 'sir madam','rusernamechecksout': 'user name checks out', 'lt3lt3lt3': 'love', 'bookmovie': 'book movie', '😀😊☺🤗😇': '😀 😊 🤗 😇',
'rpopping': 'popping','gt##': 'got to go', 'rwoooosh': 'woooosh', 'rr4r': 'redditor for redditor','ustaticsnake': 'you static snake','eli5':'explain in simple words',
'mrbeast': 'beast', 'rkarmaroulette':'karma roulette','😭😭':'😭','️\U0001f9e1💛💚💙💜':'love','🤔🤔🤔':'🤔','rgatekeeping':'gate keeping', 'moviestv': 'movie television','dinozo':'dinosaur', 'scarn': 'mayhem',
'\U0001f9e1💛💚💙💜': 'love', "pm'd":'messaged','ayyye':'yes', '\U0001f92a':'😀', 'gtno': 'get the fuck out', "walkin'": 'walking', 'rknightsofpineapple':'knights of pineapple',
'##what': 'what','rbrandnewsentence':'brand new sentence', 'bhosadiwale':'son of a bitch', 'runexpectedthanos': 'unexpected thanos',"'murica": 'america',
'udonaldduck':'you donald duck','rjobs':'jobs' ,'uitsthesnake': 'it is the snake','💕💕': '💕'}

In [ ]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else mapping[t.lower()] if t.lower() in mapping else t for t in text.split(" ")])
    return text

In [ ]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, '')
    
    specials = {'\u200b': ' ', '…': ' ', '\ufeff': ''}  # to be updated again upon checking the coverage
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

In [ ]:
def remove_newlines(sent):
  sent = re.sub(r'\s+', " ", sent )
  return sent

In [ ]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
        #x = re.sub('[0-9]', '#', x)
    return x

In [ ]:
def clean_missplets(text, mapping):
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [ ]:
# fetching data from csv file to dataframe
reddit = pd.read_csv('/content/drive/My Drive/Data/Conversations/casual_data_windows.csv')

In [ ]:
reddit.head(2)

,Unnamed: 0,0,1,2
0,0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
1,1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."


In [ ]:
# creating an empty dataframe to hold consecutive conversation-pairs
df_conv = pd.DataFrame(columns=['sent1', 'sent2'])

In [ ]:
# filling the dataframe with conversation pairs from reddit dataframe
for index, row in reddit.iterrows():
    df_conv = df_conv.append({'sent1':row["0"],'sent2':row["1"]},ignore_index=True)
    df_conv = df_conv.append({'sent1':row["1"],'sent2':row["2"]},ignore_index=True)

In [ ]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_contractions(x, contraction_mapping))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [ ]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [ ]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_contractions(x, contraction_mapping))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [ ]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x : x.lower())
df_conv['sent2'] = df_conv['sent2'].apply(lambda x : x.lower())

In [ ]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x : remove_newlines(x))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x : remove_newlines(x))

In [ ]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_numbers(x))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_numbers(x))

In [ ]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_numbers(x))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_numbers(x))

In [ ]:
df_conv['sent1'] = df_conv['sent1'].apply(lambda x: clean_missplets(x, missplet_dict))
df_conv['sent2'] = df_conv['sent2'].apply(lambda x: clean_missplets(x, missplet_dict))

In [ ]:
print(df_conv.shape)
df_conv.head()

(112594, 2)


,sent1,sent2
0,what kind of phones do you guys have,i have a pixel it is pretty great much better ...
1,i have a pixel it is pretty great much better ...,does it really charge all the way in ## min
2,i have a pixel it is pretty great much better ...,does it really charge all the way in ## min
3,does it really charge all the way in ## min,pretty fast i have never timed it but it is un...
4,does it really charge all the way in ## min,pretty fast i have never timed it but it is un...


In [ ]:
duplicateRowsDF = df_conv[df_conv.duplicated()]

In [ ]:
# removing duplicate rows
df_conv = df_conv.drop_duplicates(subset=None, keep='first', inplace=False)

In [ ]:
print(df_conv.shape)
df_conv.head()

(71052, 2)


,sent1,sent2
0,what kind of phones do you guys have,i have a pixel it is pretty great much better ...
1,i have a pixel it is pretty great much better ...,does it really charge all the way in ## min
3,does it really charge all the way in ## min,pretty fast i have never timed it but it is un...
5,pretty fast i have never timed it but it is un...,cool i have been thinking of getting one my ph...
6,what kind of phones do you guys have,samsung galaxy j1 it is my first cell phone an...


In [ ]:
# filtering out unique sentence set from conversation pairs. this is needed to be done before creating word to index dictionary

encoder_data = df_conv['sent1']
decoder_data = df_conv['sent2']

In [ ]:
# Series of unique sentences
unique_sents = decoder_data.append(encoder_data[:1])
len(unique_sents)

71053

# Embedding

In [ ]:
sentences_tok = []
for sent_str in unique_sents.values:
    tokens = sent_str.split()
    sentences_tok.append(tokens)

In [ ]:
sentences_tok[5:6]


[['what',
  'do',
  'you',
  'think',
  'of',
  'it',
  'anything',
  'you',
  'do',
  'not',
  'like']]

In [ ]:
from gensim.models import FastText
model_fast = FastText(sentences_tok, size=300, window=5, min_count=1, workers=4,sg=1)

In [ ]:
model_fast.wv['you']

array([-1.89220861e-01,  2.94760287e-01, -4.83378410e-01, -6.67391792e-02,
       -2.98199743e-01, -5.30537605e-01,  2.86983280e-03, -2.20747218e-01,
        2.91514486e-01, -1.10376872e-01, -2.48871483e-02, -5.25510497e-02,
       -7.50873908e-02,  1.08912639e-01, -3.52990597e-01,  1.25542417e-01,
        5.85989170e-02,  1.82362810e-01,  2.66778208e-02, -1.18527018e-01,
       -2.25809649e-01,  1.96421370e-01,  8.64098407e-03,  1.30207185e-02,
        3.52608897e-02,  2.62196194e-02, -4.14143711e-01, -2.66967803e-01,
       -1.36522442e-01, -5.65856881e-02, -2.40269825e-01, -7.51627684e-02,
        5.58274053e-02,  5.17799929e-02,  1.88356787e-01, -3.00463527e-01,
       -8.28464255e-02,  9.94886607e-02, -5.72060347e-02,  6.98634461e-02,
        2.45434901e-04,  1.71174537e-02, -1.80396009e-02,  3.39121580e-01,
        2.86912441e-01,  7.77478069e-02,  2.44166225e-01, -1.68209314e-01,
        3.04887682e-01, -2.97264308e-01, -1.29068613e-01,  2.90176570e-01,
       -4.01426256e-01, -

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(unique_sents.values))

In [ ]:
# word to index dictionary is created using keras Tokenizer class
word_index = tokenizer.word_index

In [ ]:
# Writing the custom embedding to a text file from which torchtext will read the embeddings and them into vector object later

with open('/content/drive/My Drive/Data/custom_fasttext_embeddings.txt', 'w+') as f:
  for token, vector in word_index.items():
    vector = model_fast.wv[token]
    vector_str = ' '.join([str(v) for v in vector])
    f.write(f'{token} {vector_str}\n')

# Torchtext

In [ ]:
# spaCy has model for each language which is needed to be loaded so we can access the tokenizer of that language's model

# downloading english language model

!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
english = spacy.load('en')

In [ ]:
def tokenize_sent1(sentence):
  #return [tok.text for tok in english.tokenizer(sentence)]  #not reversing
  return [tok.text for tok in english.tokenizer(sentence)][::-1]

def tokenize_sent2(sentence):
  return [tok.text for tok in english.tokenizer(sentence)]

In [ ]:
# batch_first: Whether to produce tensors with the batch dimension first.
# by default batch_first is False, but the transformer should be fed examples with the batch first ([batch_size, seq_len]) 
# thus making the attribute value True

sent1 = Field(tokenize = tokenize_sent1,init_token='<sos>',eos_token='<sos>',batch_first = True)
sent2 = Field(tokenize = tokenize_sent2,init_token='<sos>',eos_token='<sos>',batch_first = True) 

In [ ]:
train_data , test_data = train_test_split(df_conv,test_size = 0.2)

In [ ]:
%cd '/content/drive/My Drive/Data/Conversations/'

/content/drive/My Drive/Data/Conversations


In [ ]:
train_data.to_csv('train.csv',index = False)
test_data.to_csv('test.csv',index = False)

In [ ]:
data_fields = [('sent1',sent1),('sent2',sent2)]

In [ ]:
# TabularDataset(Dataset):Defines a Dataset of columns stored in CSV, TSV, or JSON format.
# Create a TabularDataset given a path, file format, and field list

train_data , test_data = TabularDataset.splits(path ='/content/drive/My Drive/Data/Conversations/' ,train='train.csv', test ='test.csv', format='csv', fields=data_fields)

In [ ]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(test_data.examples)}")

Number of training examples: 56842
Number of validation examples: 14212


In [ ]:
print(vars(train_data.examples[1]))

{'sent1': ['food', 'have', 'to', 'deserve', 'polite', 'are', 'who', 'people', 'only', 'do', 'why'], 'sent2': ['fuck', 'off', 'with', 'your', 'white', 'knight', 'bullshit']}


In [ ]:
print(vars(test_data.examples[1]))

{'sent1': ['username', 'your', 'behind', 'story', 'the', 'is', 'what'], 'sent2': ['i', 'got', 'this', 'from', 'that', 'stupid', 'got', "'", 'eeeem', 'vine']}


In [ ]:
import torchtext.vocab as vocab

# reading the custom embedding wriiten on a text file with TorchText and converting them to vector objects

custom_embeddings = vocab.Vectors(name = '/content/drive/My Drive/Data/custom_fasttext_embeddings.txt')
sent1.build_vocab(train_data, vectors = custom_embeddings) 
sent2.build_vocab(train_data, vectors = custom_embeddings)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cpu')

In [ ]:
BATCH_SIZE = 150

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, test_data),
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)

In [ ]:
print(len(sent1.vocab))
print(len(sent2.vocab))

20891
24185


# Building the model

In [ ]:
# Encoder

class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)

        # step added for custom embedding
        # loading the pre-trained embeddings into the model 
        self.tok_embedding.weight.data.copy_(sent1.vocab.vectors)

        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [ ]:
# Encoder Layer

class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, src len]
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [ ]:
# MultiHead Attention Layer

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [ ]:
# Position-wise Feedforward Layer

class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [ ]:
# Decoder

class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        
        # step added for custom embedding
        self.tok_embedding.weight.data.copy_(sent2.vocab.vectors)

        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, trg len]
        #src_mask = [batch size, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [ ]:
# Decoder Layer

class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, trg len]
        #src_mask = [batch size, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [ ]:
# Seq2seq

class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [ ]:
# Training the model


INPUT_DIM = len(sent1.vocab)
OUTPUT_DIM = len(sent2.vocab)
HID_DIM = 300
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 10
DEC_HEADS =10
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [ ]:
SENT1_PAD_IDX = sent1.vocab.stoi[sent1.pad_token]  #Vocab.stoi – A collections.defaultdict instance mapping token strings to numerical identifiers
SENT2_PAD_IDX = sent2.vocab.stoi[sent2.pad_token]

model = Seq2Seq(enc, dec, SENT1_PAD_IDX, SENT2_PAD_IDX, device).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 25,970,357 trainable parameters


In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
model.apply(initialize_weights);

In [ ]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = SENT2_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        #src = batch.src
        #trg = batch.trg
        sent1 = batch.sent1
        sent2 = batch.sent2
        
        optimizer.zero_grad()
        
        output, _ = model(sent1, sent2[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        sent2 = sent2[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, sent2)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            sent1 = batch.sent1
            sent2 = batch.sent2

            output, _ = model(sent1, sent2[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            sent2 = sent2[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, sent2)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 48m 29s
	Train Loss: 5.906 | Train PPL: 367.206
	 Val. Loss: 5.326 |  Val. PPL: 205.673
Epoch: 02 | Time: 49m 23s
	Train Loss: 5.068 | Train PPL: 158.779
	 Val. Loss: 5.155 |  Val. PPL: 173.328
Epoch: 03 | Time: 49m 59s
	Train Loss: 4.720 | Train PPL: 112.126
	 Val. Loss: 5.114 |  Val. PPL: 166.337
Epoch: 04 | Time: 49m 38s
	Train Loss: 4.429 | Train PPL:  83.819
	 Val. Loss: 5.125 |  Val. PPL: 168.116
Epoch: 05 | Time: 49m 38s
	Train Loss: 4.160 | Train PPL:  64.046
	 Val. Loss: 5.160 |  Val. PPL: 174.146
